# Starting Spark Session

In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("read-postgres")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-42.4.0.jar")
         .getOrCreate())
sc = spark.sparkContext

## Read Postgres

In [2]:
df_movies = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://mypostgres:5432/test")
    .option("dbtable", "public.movies")
    .option("user", "airflow")
    .option("password", "airflow")
    .load()
)

In [3]:
df_ratings = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://mypostgres:5432/test")
    .option("dbtable", "public.ratings")
    .option("user", "airflow")
    .option("password", "airflow")
    .load()
)

# Top 10 movies with more ratings

In [4]:
df_movies = df_movies.alias("m")
df_ratings = df_ratings.alias("r")

df_join = df_ratings.join(df_movies, df_ratings.movieId == df_movies.movieId).select("r.*","m.title")

In [5]:
from pyspark.sql import functions as F

df_result = (
    df_join
    .groupBy("title")
    .agg(
        F.count("timestamp").alias("qty_ratings")
        ,F.mean("rating").alias("avg_rating")
    )
    .sort(F.desc("qty_ratings"))
    .limit(10)
)

In [7]:
df_result.coalesce(1).write.format("csv").mode("overwrite").save("/home/jovyan/work/resources/data/output_postgres", header=True)